In [58]:
import pandas as pd
df = pd.read_csv('spam.csv', encoding='latin-1')
df = df.iloc[:, :2]
df = df.replace('ham', 0).replace('spam', 1)
df = df.rename(columns = {'v1': 'label', 'v2': 'message'})
df = df.drop_duplicates().reset_index(drop=True)

1. This code reads the table into a dataframe and encodes it with latin-1, then removes all but the first two columns of data. After that the dataframe is categorically encoded so that "ham" is now 0 and "spam" 1. The column names are changed from v1 and v2 to label and message. Rows e and f are removed in the data where e = \[label,message\] and f = \[label1,message1\] if label=label1 and message=message1. The index is then reset so that the difference between the index of two adjacent rows is +/- 1. The "drop=True" prevents the old index from becoming a new column in the dataframe with the reset index.

Overall this bit of code is essentially preparing the data to be usable/useful to us/any models going forward (pre-processing)

In [59]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
messages = df['message']
y = df['label']
v = CountVectorizer()
x = v.fit_transform(messages)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2021)

2. Tokenizing the data means that you split the text data into separate pieces. They can vary in size, for example you could tokenize data into phrases, sentences, words, etc. 

Tokens are defined as any word larger than one character by CountVectorizer with its default parameters. CountVectorizer also excludes punctuation

There are 8672 unique tokens in the dataset (found with x.shape, the column count is the number of unique tokens)

The training dataset does not contain at least one instance of each token found with x_train.toarray().shape == x_train.toarray()\[:,(x_train.toarray() != 0).any(axis=0)\].shape
which compares the original training set to the training set with any columns with all 0 values removed if they were the same shape then each column (or token) in the training set would have at least 1 data point that contained that token.

In [60]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import precision_score, recall_score
mnb_clf = MultinomialNB()
mnb_clf.fit(x_train, y_train)
y_hat = mnb_clf.predict(x_test)
mnbScores = [mnb_clf.score(x_test, y_test), precision_score(y_test, y_hat), recall_score(y_test, y_hat)]

3. tp = # of true positive, tn = # of true negative, fp = # of false positive, fn = # of false negative

# Accuracy: 
$$\frac{tp+tn}{(tp+tn+fp+fn)}$$ 
This is the probability of a correct classification. You want this to be a relatively high value when choosing a model in most all cases, regardless of what data the model is for.

# Precision: 
$$\frac{tp}{(tp+fp)}$$ 
This is the probability of a model to not mislabel a true positive (probability that the model correctly predicts a positive).
If its important for the model to have a minimized chance of predicting a false positive than precision should be a priority and a high percentage.

# Recall: 
$$\frac{tp}{(tp+fn)}$$ 
This is the probability of a model to find all positive cases (probability that the model will predict positive when the known classification is as well).
If its important for the model to find all positive classifications than recall should be a priority and have a larger value.

You'd ideally want all three metrics to be large as that means your model performs well overall.

In [61]:
from sklearn.svm import LinearSVC
Lsvc_clf = LinearSVC()
Lsvc_clf.fit(x_train, y_train)
y_hatSVC = Lsvc_clf.predict(x_test)
lsvcScores = [Lsvc_clf.score(x_test, y_test), precision_score(y_test, y_hatSVC), recall_score(y_test, y_hatSVC)]

In [62]:
from sklearn.linear_model import LogisticRegression
reg_clf = LogisticRegression()
reg_clf.fit(x_train, y_train)
y_hatReg = reg_clf.predict(x_test)
regScores = [reg_clf.score(x_test, y_test), precision_score(y_test, y_hatReg), recall_score(y_test, y_hatReg)]

|	 |Accuracy 	        |Precision 		    |Recall             |
|----|------------------|-------------------|-------------------|
|MNB |0.9777562862669246| 0.9154929577464789| 0.9219858156028369|
|SVC |0.9816247582205029| 0.9919354838709677| 0.8723404255319149|
|REG |0.9787234042553191| 0.983739837398374 | 0.8581560283687943|

6. I would use the linear support vector machine in practice because of the nature of the data and categories. The purpose of the models are to separate between spam and not spam text. You wouldn't want to not receive a real message due to the model saying labelling it spam, thus you'd want to reduce the number of false positives. This means a model with high precision. You'd obviously also want a model with high accuracy, but all three models have relatively good accuracy scores with the highest (linear svm) being only ~1% more than the other two. The Recall percentage wouldn't matter as much for this problem since obtaining ALL positive cases (spam) isn't massively important if it comes along with a diminished ability to prevent false positives. Receiving some spam is better than not receiving a real message. Even if these scores were given without the context of the data, I'd still probably use the linear svm in practice since it has the highest accuracy and precision as well as the second highest recall scores.

Nothing major really surprised me about the results. It was at first strange to see the three scores moving independent of each other, but after looking at the confusion matrices for each model it made sense how this could occur.

1. The data is fairly old considering how quickly things change especially with how connected the world is due to the internet and social media. The data could be outdated thus leading to incorrect classification when exposed to more modern SMS syntax.
2. The data collected is from mainly British messages and Singaporean University Students messages. This could result in poorer performance if the TMobile Spam filter is used on messages from other origins like North American (which is where TMobile has the most users).
3. The Dataset is also relatively small after removing duplicates (only ~5,000 data points). TMobile has well over 100 million users given that its a international company and the number of US users is 104.79 million. (https://www.statista.com/statistics/219577/total-customers-of-t-mobile-usa-by-quarter/). I don't think that the dataset would accurately represent the nature of what could be millions of messages and thus performance of the model would drop if you attemped to use such a small dataset to try and make assumptions about a significantly larger and more diverse set of messages.

Information about the dataset (spam.csv) was obtained from https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection